### Data Acquisition

In [1]:
#DATA ACQUISITION - DATABASE

In [1]:
# all modules
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from sqlalchemy import inspect, create_engine
import sqlsoup

In [2]:
%load_ext sql
%config SqlMagic.autocommit = False

In [3]:
#load data
%sql sqlite:////home/carmencuadrado/Ironhack/Project-1/data-project-template/data/raw/raw_data_project_m1.db

In [4]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:////home/carmencuadrado/Ironhack/Project-1/data-project-template/data/raw/raw_data_project_m1.db
Done.
Returning data to local variable tables


In [5]:
tables.DataFrame()

,name
0,personal_info
1,country_info
2,career_info
3,poll_info


In [6]:
%%sql left <<

SELECT *
FROM country_info
LEFT JOIN career_info ON career_info.uuid = country_info.uuid 
LEFT JOIN personal_info ON personal_info.uuid = country_info.uuid
LEFT JOIN poll_info ON poll_info.uuid = country_info.uuid;

 * sqlite:////home/carmencuadrado/Ironhack/Project-1/data-project-template/data/raw/raw_data_project_m1.db
Done.
Returning data to local variable left


In [7]:
jobs_db=left.DataFrame()
jobs_db.isnull().sum()

uuid                                                      0
country_code                                              0
rural                                                     0
uuid                                                      0
dem_education_level                                     663
dem_full_time_job                                         0
normalized_job_code                                    3947
uuid                                                      0
age                                                       0
gender                                                    0
dem_has_children                                          0
age_group                                                 0
uuid                                                      0
question_bbi_2016wave4_basicincome_awareness              0
question_bbi_2016wave4_basicincome_vote                   0
question_bbi_2016wave4_basicincome_effect                 0
question_bbi_2016wave4_basicincome_argum

In [8]:
jobs_db['normalized_job_code'].isnull().sum()

3947

In [9]:
# Store database in a csv to ease its readiness
jobs_csv=jobs_db.to_csv(r'../data/processed/jobs_db.csv', index=False)

In [10]:
#DATA ACQUISITION - API

In [13]:
response = requests.get(f'http://api.dataatwork.org/v1/jobs/autocomplete?contains=data')
json_data=response.json()
api_df=pd.DataFrame(json_data)
api_df.shape

(156, 4)

In [14]:
api_df = pd.DataFrame(json_data)
api_df=api_df.to_json(r'../data/processed/api_data.json')

In [15]:
#DATA ACQUISITION - WEB SCRAPPING

In [16]:
url='https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html=requests.get(url).content

In [17]:
html

b'\xef\xbb\xbf<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta http-equiv="X-UA-Compatible" content="Ie=Edge,chrome=1"/> <meta charset="UTF-8" /><title>Glossary:Country codes - Statistics Explained</title>\n<meta name="generator" content="MediaWiki 1.22.5" />\n<link rel="shortcut icon" href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" />\n<link rel="search" type="application/opensearchdescription+xml" href="/eurostat/statistics-explained/opensearch_desc.php" title="Statistics Explained (en)" />\n<link rel="EditURI" type="application/rsd+xml" href="https://ec.europa.eu/eurostat/statistics-explained/api.php?action=rsd" />\n<link rel="alternate" type="application/atom+xml" title="Statistics Explained Atom feed" href="/eurostat/statistics-explained/index.php?title=Special:RecentChanges&amp;feed=atom" />\n<link rel="stylesheet" href="https://ec.europa.eu/eurostat/statistics-explained/load.php?debug=true&amp;lang=en&amp;modules=ext.rtlcite&amp;

In [ ]:
soup = BeautifulSoup(html, 'lxml')
countries = soup.find_all('td')
countries